# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd
from importlib import reload

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=False)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [ ]:
config = liriscat.utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.001
config['inner_user_lr'] = 0.016848380924625605
config['lambda'] = 9.972254466547545e-06

meta_trainers = ['MAML']
config['meta_lr'] = 0.001
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12

#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 21:26] math2


In [5]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 21:26] #### math2 ####
[INFO 21:26] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'd_in': 4} ####


In [6]:

for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

[INFO 21:27] #### meta_trainer : MAML ####
[INFO 21:27] #### i_fold : 1 ####


[INFO 21:38] #### seed : 0 ####
[INFO 21:38] Random_cont_model
[INFO 21:38] compiling CDM model
[INFO 21:40] compiling selection model
[INFO 21:40] ------- START Training
[INFO 21:40] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 21:42] ----- User batch : 0
[INFO 21:47] ----- User batch : 1
[INFO 21:51] ----- User batch : 2
[INFO 21:54] ----- User batch : 3
[INFO 21:58] ----- User batch : 4
[INFO 22:00] - learning_users_emb gn: 141.30142211914062
[INFO 22:00] - learning_users_emb norm: 2.273587942123413
[INFO 22:00] - learning_users_emb: 0.5
[INFO 22:07] rmse : 0.6865495443344116
[INFO 22:07] valid_rmse : 0.8779405951499939
[INFO 22:07] valid_loss : 1.1226927042007446


  1%|          | 1/100 [00:25<42:08, 25.54s/it]

...

[INFO 41:28] ----- User batch : 0
[INFO 41:31] ----- User batch : 1
[INFO 41:35] ----- User batch : 2
[INFO 41:38] ----- User batch : 3
[INFO 41:42] ----- User batch : 4
[INFO 41:43] - learning_users_emb gn: nan
[INFO 41:43] - learning_users_emb norm: nan
[INFO 41:43] - learning_users_emb: 0.000244140625
[INFO 41:48] rmse : 1.5365015268325806
[INFO 41:48] valid_rmse : 1.9648356437683105
[INFO 41:48] valid_loss : nan


 56%|█████▌    | 56/100 [20:07<15:48, 21.56s/it]

[INFO 41:48] -- END Training --
[INFO 41:48] train on cuda
Learning users emb shape: torch.Size([3911, 16])



100%|██████████| 6/6 [00:04<00:00,  1.31it/s]


[INFO 42:00] ({0: {'mi_acc': 0.651021420955658}, 1: {'mi_acc': 0.651021420955658}, 2: {'mi_acc': 0.651021420955658}, 3: {'mi_acc': 0.651021420955658}, 4: {'mi_acc': 0.651021420955658}, 5: {'mi_acc': 0.651021420955658}}, {0: {'meta_doa': 0.5296872042563507}, 1: {'meta_doa': 0.5415014669996929}, 2: {'meta_doa': 0.5458948279149827}, 3: {'meta_doa': 0.5457547038735773}, 4: {'meta_doa': 0.54543875857391}, 5: {'meta_doa': 0.5429904581351386}})
[INFO 42:00] 0.037946762771058316


In [7]:
S.best_epoch

36

In [13]:

for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()

[INFO 27:16] #### seed : 0 ####
[INFO 27:16] Random_cont_model
[INFO 27:16] compiling CDM model
[INFO 27:16] compiling selection model


[INFO 27:16] ------- START Training
[INFO 27:16] train on cuda


/home/arthur/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 27:17] ----- User batch : 0
[INFO 27:20] ----- User batch : 1
[INFO 27:22] ----- User batch : 2
[INFO 27:25] ----- User batch : 3
[INFO 27:28] ----- User batch : 4
[INFO 27:30] - learning_users_emb gn: 138.84461975097656
[INFO 27:30] - learning_users_emb norm: 2.273587942123413
[INFO 27:30] - learning_users_emb: 0.5
[INFO 27:34] rmse : 0.6865495443344116
[INFO 27:34] valid_rmse : 0.8779405951499939
[INFO 27:34] valid_loss : 1.123011589050293


  1%|          | 1/100 [00:17<28:41, 17.39s/it]

...

[INFO 40:25] ----- User batch : 0
[INFO 40:28] ----- User batch : 1
[INFO 40:31] ----- User batch : 2
[INFO 40:34] ----- User batch : 3
[INFO 40:36] ----- User batch : 4
[INFO 40:38] - learning_users_emb gn: 14.411944389343262
[INFO 40:38] - learning_users_emb norm: 0.41957318782806396
[INFO 40:38] - learning_users_emb: 0.00390625
[INFO 40:42] rmse : 0.6066398620605469
[INFO 40:42] valid_rmse : 0.7757542729377747
[INFO 40:42] valid_loss : 0.9387263655662537


 45%|████▌     | 45/100 [13:25<16:24, 17.90s/it]

[INFO 40:42] -- END Training --
[INFO 40:42] train on cuda



100%|██████████| 6/6 [00:04<00:00,  1.42it/s]


0.07389431460485107
({0: {'mi_acc': 0.6615844368934631}, 1: {'mi_acc': 0.6615844368934631}, 2: {'mi_acc': 0.6615844368934631}, 3: {'mi_acc': 0.6615844368934631}, 4: {'mi_acc': 0.6615844368934631}, 5: {'mi_acc': 0.6615844368934631}}, {0: {'meta_doa': 0.5284317794068907}, 1: {'meta_doa': 0.5480420894515283}, 2: {'meta_doa': 0.5676536835446419}, 3: {'meta_doa': 0.5865667732627711}, 4: {'meta_doa': 0.6061013953574441}, 5: {'meta_doa': 0.6205151192129748}})
